In [27]:
%run ItemModel.ipynb
%run APIService.ipynb

In [11]:
import inspect, re
import pandas as pd
import csv

In [31]:
class GetData:
    newest = 0
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
    APIService = APIService(newest)
    i = 1
    def __init__(self, newest):
        self.newest = newest
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def getData(self):
        while True:
            try:
                self.APIService.thoiTrangNuPhuKien(self.newest)
                json_pydata = self.APIService.get()
                print(len(json_pydata))
                print(f'get! {self.i}')
                self.items_json.extend(json_pydata['items'])
        
                self.newest += 50
                self.i += 1
        
                time.sleep(3)
            except Exception as e:
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.historical_sold = item['historical_sold']
            itemModel.sold = item['sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            for tier_variation in item['tier_variations']:
                itemModel.options = len(tier_variation['options'])
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self):
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'name\tshop_location\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\trating_count\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discoun\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'sold\thistorical_sold\n')

        for item in self.all_items:
                file.write(f'{item.name}\t{item.shop_location}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.rating_count}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.sold}\t{item.historical_sold}\n')
    
    pass

In [45]:
getData = GetData(0)

In [46]:
getData.getData()


17
get! 1
Error: Expecting value: line 1 column 1 (char 0)


In [34]:
getData.parseData()

In [35]:
# Test ParseData
getData.all_items[10].name

'Gọng kính cận tròn kim loại thời trang nữ 2736 nhiều màu'

In [36]:
getData.saveFile()

In [44]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df.head()


,name,shop_location,shopee_verified,discount,is_adult,options,show_official_shop_label_in_title,rating_star,rating_count,flash_sale,...,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,sold,historical_sold
0,[Mã FACB201 hoàn 15% tối đa 30K xu đơn 50K] Kẹp tóc ngọc trai thanh lịch thời trang Hàn Quốc cho nữ nhiều kiểu tùy chọn,Nước ngoài,True,33%,False,19,False,4.940490,"[857, 4, 2, 6, 17, 828]",None,...,None,534,45090,857,False,False,9,1,2684,3899
1,[Mã FACB201 hoàn 15% tối đa 30K xu đơn 50K] Set 6 Kẹp Tóc Kiểu Dáng Thời Trang Phong Cách Hàn Quốc Cho Nữ,Nước ngoài,True,40%,False,12,False,4.840000,"[50, 1, 0, 1, 2, 46]",None,...,None,128,9291,50,False,False,6,1,158,221
2,Vòng Cổ Choker Thời Trang Cá Tính Cho Nữ,Nước ngoài,True,25%,False,1,False,5.000000,"[7, 0, 0, 0, 0, 7]",None,...,None,18,1027,7,False,False,9,0,16,21
3,[Mã FACB201 hoàn 15% tối đa 30K xu đơn 50K] Kẹp Tóc Nữ Trang Sức Hình Gấu / Thỏ Dễ Thương Hợp Thời Trang,Nước ngoài,True,41%,False,2,False,4.733333,"[15, 1, 0, 0, 0, 14]",None,...,None,14,579,15,False,False,9,0,18,42
4,Nữ Thời Trang Tinh Tế Vòng tay mạ bạc thiết kế nhiều kiểu dáng thời trang xinh xắn cho nữ,Nước ngoài,True,49%,False,9,False,4.905660,"[106, 2, 0, 0, 2, 102]",None,...,None,65,4054,106,False,False,9,0,184,417
